# Описание проекта

Для коммуникационной компании, оказывающей услуги на территории СНГ, необходимо определить уровень лояльности клиентов на территории России (NPS) и подготовить дашборд на основе обратной связи, полученной в ходе анкетирования. Результаты опроса были занесены в базу данных, содержащую сведения о геолокации клиента, некоторых личных данных, позволяющих разбить клиентов на группы, и времени жизни клиента в компании. Данные были предоставлены в SQLite, поэтому требуют проверки и предварительной обработки.

## Ознакомление с данными.

Данные состоят из нескольких таблиц, связанных между собой ключами. Необходимо ознакомиться с каждой таблицей, проверить её на удобство формата данных и загрузить необходимые столбцы из разных таблиц с помощью запроса на SQL в единую таблицу для дальнейшей обработки в Tableau.

In [1]:
# Импортируем необходимые библиотеки:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
# Определим путь считывания файла:

# с локального расположения
path_to_db_local = '.db'
# с сервера Практикума
path_to_db_platform = '.db'
path_to_db = None

# пропишем условия загрузки файла при разных ситуациях:
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
# Посмотрим на певую таблицу, загрузив её через запрос SQL:
query = """
SELECT *
FROM user
"""
user = pd.read_sql(query, engine)
user.head(3)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10


В таблице user собраны данные о пользователях. Однако представленная форма данных не очевидна и нуждается в дополнении данными других таблиц путём присоединения их по первичным ключам. Проверим, какие данные в других таблицах стоит выгрузит в закодированные поля.

In [4]:
# Проверим данные таблицы геолокации:
query = """
SELECT *
FROM location
"""
loc = pd.read_sql(query, engine)
loc.head(3)

,location_id,city,country
0,1,Архангельск,Россия
1,2,Астрахань,Россия
2,3,Балашиха,Россия


Чтобы убедиться в отсутствии дубликатов, необходимо проверить нет ли ошибок в записи названия городов и страны.

In [5]:
# Выгрузим уникальные значения названий городов:
query = """
SELECT DISTINCT(city)
FROM location
"""
city = pd.read_sql(query, engine)
city['city'].to_list()

['Архангельск',
 'Астрахань',
 'Балашиха',
 'Барнаул',
 'Белгород',
 'Брянск',
 'Владивосток',
 'Владимир',
 'Волгоград',
 'Волжский',
 'Воронеж',
 'Грозный',
 'Екатеринбург',
 'Иваново',
 'Ижевск',
 'Иркутск',
 'Казань',
 'Калининград',
 'Калуга',
 'Кемерово',
 'Киров',
 'Краснодар',
 'Красноярск',
 'Курск',
 'Липецк',
 'Магнитогорск',
 'Махачкала',
 'Москва',
 'НабережныеЧелны',
 'НижнийНовгород',
 'НижнийТагил',
 'Новокузнецк',
 'Новосибирск',
 'Омск',
 'Оренбург',
 'Пенза',
 'Пермь',
 'РостовнаДону',
 'Рязань',
 'Самара',
 'СанктПетербург',
 'Саранск',
 'Саратов',
 'Смоленск',
 'Сочи',
 'Ставрополь',
 'Сургут',
 'Тверь',
 'Тольятти',
 'Томск',
 'Тула',
 'Тюмень',
 'УланУдэ',
 'Ульяновск',
 'Уфа',
 'Хабаровск',
 'Чебоксары',
 'Челябинск',
 'Череповец',
 'Чита',
 'Якутск',
 'Ярославль']

In [6]:
# Выгрузим уникальные значения стран:
query = """
SELECT DISTINCT(country)
FROM location
"""
country = pd.read_sql(query, engine)
country['country'].to_list()

['Россия']

В названиях городов и страны ошибок нет. Написание городов не очень удобно для восприятия, однако исключает возможность ошибочного написания. Для анализа по геолокации пользователей оба столбца можно взять без изменений.

In [7]:
# Проверим данные таблицы возрастных характеристик:
query = """
SELECT *
FROM age_segment
"""
age = pd.read_sql(query, engine)
age

,age_gr_id,bucket_min,bucket_max,title
0,1,NaN,15.0,01 до 16
1,2,16.0,24.0,02 16-24
2,3,25.0,34.0,03 25-34
3,4,35.0,44.0,04 35-44
4,5,45.0,54.0,05 45-54
5,6,55.0,64.0,06 55-64
6,7,66.0,NaN,07 66 +
7,8,NaN,NaN,08 n/a


Для анализа в качестве условия группировки удобно будет взять данные столбца title, однако при ивыгрузке стоит отбросить первые 4 символа порядкового номера и оставить лишь название границ интервалов.

In [8]:
# Проверим данные таблицы трафика
query = """
SELECT *
FROM traffic_segment
"""
traffic = pd.read_sql(query, engine)
traffic

,tr_gr_id,bucket_min,bucket_max,title
0,1,0.00,0.00,01 0
1,2,0.00,0.01,01 0-0.01
2,3,0.01,0.10,02 0.01-0.1
3,4,0.10,1.00,03 0.1-1
4,5,1.00,5.00,04 1-5
5,6,5.00,10.00,05 5-10
6,7,10.00,15.00,06 10-15
7,8,15.00,20.00,07 15-20
8,9,20.00,25.00,08 20-25
9,10,25.00,30.00,09 25-30


Данные трафика также удобно разбиты на группы в последнем столбце, в котором при выгрузки из таблицы следует провести аналогичные возрасту манипуляции.

In [9]:
# Проверим данные таблицы длительности "жизни" клиента:
query = """
SELECT *
FROM lifetime_segment
"""
lifetime = pd.read_sql(query, engine)
lifetime

,lt_gr_id,bucket_min,bucket_max,title
0,1,1.0,1.0,01 1
1,2,2.0,2.0,02 2
2,3,3.0,3.0,03 3
3,4,4.0,6.0,04 4-6
4,5,7.0,12.0,05 7-12
5,6,13.0,24.0,06 13-24
6,7,25.0,36.0,07 25-36
7,8,36.0,NaN,08 36+


Данные таблицы длительности существования клиента следует обработать и выгрузить аналогично таблицам с возрастными и характеристиками и трафиком.

## Выгрузка базы данных

После определения столбцов для выгрузки в каждой таблице можно подготовить общую таблицу для последующей обработки и посторения графиков. Отсутствующие поля создадим заполним согласно условиям, присоединение таблиц проведём слева, определив таблицу user как главную. 

In [10]:
# Создадим общий запрос для вывода из всех таблиц интересующих столбцов в удобном виде:
query = """
SELECT u.user_id,
        u.lt_day,
        CASE
            WHEN u.lt_day <= 365 THEN 'new'
            ELSE 'old'
        END AS is_new,
        u.age,
        CASE
            WHEN u.gender_segment = 0.0 THEN 'male'
            WHEN u.gender_segment = 1.0 THEN 'female'
            ELSE 'other'
        END AS gender_segment,
        u.os_name,
        u.cpe_type_name,
        l.country,
        l.city,
        SUBSTRING(a_s.title, 4) AS age_segment,
        SUBSTRING(t_s.title, 4) AS traffic_segment,
        SUBSTRING(l_s.title, 4) AS lifetime_segment,
        u.nps_score,
        CASE
            WHEN u.nps_score >= 9 THEN 'promoters'
            WHEN u.nps_score <= 6 THEN 'detractors'
            ELSE 'passives'
        END AS nps_group
FROM user AS u
    LEFT JOIN location AS l ON l.location_id = u.location_id
    LEFT JOIN age_segment AS a_s ON a_s.age_gr_id = u.age_gr_id
    LEFT JOIN traffic_segment AS t_s ON t_s.tr_gr_id = u.tr_gr_id
    LEFT JOIN lifetime_segment AS l_s ON l_s.lt_gr_id = u.lt_gr_id
"""

In [11]:
data = pd.read_sql(query, engine)
# Ознакомимся с несколькими строками, проверив удобство формата данных:
data.head(5)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,45.0,female,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,2344,old,53.0,male,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,467,old,57.0,male,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,promoters
3,A004TB,4190,old,44.0,female,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,promoters
4,A004XT,1163,old,24.0,male,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,promoters


In [12]:
# Запишем таблицу в отдельный файл для дальнейшей обработки и визуализации:

data.to_csv('.csv', index=False)

## Вывод

Для коммуникационной компании, оказывающей услуги на территории СНГ, необходимо определить уровень лояльности клиентов на территории России (показатель NPS) и подготовить дашборд на основе обратной связи, полученной в ходе анкетирования. Результаты опроса были занесены в базу данных, содержащую сведения о геолокации клиента, некоторых личных данных, позволяющих сегментировать характеристики клиентов, и времени жизни клиента в компании.

NPS или индекс потребительской лояльности показывает, насколько клиенты довольны услугами компании. Подсчёт производится в виде разницы процентных долей лояльных клиентов (оценка 9 и 10 в опросе) и нелояльных (оценка от 0 до 6 в опросе). Полученные от заказчика данные были сгруппированы из нескольких таблиц в одну с удобными для сегментации клиентов полями. Для более точного подсчета и группировки из анализируемой базы были исключены данные с отсутсвующими значениями пола и возраста (2,55% исходных данных), заменены значения пола с числовых на строковые и добавлены столбцы сегментов возраста, трафика, lifetime и NPS-статуса.

Общий NPS компании составляет 21,97. Это говорит о том, что доля критиков высока и компании следует скорректировать курс развития в области работы с клиентами.

- Количество участников опроса прямо пропорционально населённости города. Большое количество клиентов из мегаполисов с невысоким уровнем лояльности снижают общий NPS. Непосредственной корреляции между геолокацией и уровнем NPS не замечено, поэтому для анализа его колебаний следует обратить внимание на другие факторы. Наименее лояльными оказались клиенты как в крупных городах, таких как Воронеж (4,46), Тольятти (6,07), Тюмень (8,10), Хабаровск (9,35), так и в в небольших, например, Рязань(7,07), Тверь (7,97). Наиболее лояльны клиенты таких городов как Иваново (46,92), Череповец (44,59), Саранск (43,82).


- Наиболее активно принимали участие в опросе клиенты возрастом 25-55 лет, что соответствует активному рабочему периоду жизни.  Клиенты старше 45 лет более всего расположены рекомендовать компанию. Клиенты моложе 25 лет активны во всех сегментах трафика, но имеют наименьший показатель NPS. Среди них зафиксировано наибольшее количество пользователей iOS, которые также негативно настроены. Компании следует обратить внимание на эту категорию: возможно сложности или неудобство использования програмного обеспечения на этой операционной системе негативно сказываются на мнении пользователей.


- Большинство респондентов - женщины (54%), которые чаще готовы рекомендовать компанию, чем мужчины. Среди недовольных и нейтральных клиентов распределение мужчин и женщин примерно равно. В Москве мужчин преобладающее большинство, в Санкт-Петербурге и Самаре немногим больше женщин. В остальных городах ситуация обратная или равная, что может повышать уровень NPS, так как женщины более лояльны. 


- Среди новых клиентов уровень лояльности выше, нежели среди знакомых  с компанией более года, что подтверждается существенным ростом показателя NPS при уменьшении показателя lifetime. 

По итогам расчётом можно утверждать, что наиболее лояльны люди среднего и пожилого возраста, быстро получающие необходимую услугу, чаще женщины, особенно будучи новыми клиентами. Наименее расположены к компании молодые клиенты, пользователи iOS, являющиеся пользователшями услуг более двух лет.

Более детально ознакомиться с показателями можно в презентации ресурса Tableau:

https://public.tableau.com/views/NPS-project/NPS_1?:language=en-US&:sid=&:display_count=n&:origin=viz_share_link

Также с показателями можно ознакомить в презентации:

https://disk.yandex.ru/i/cZZWaYpBB_zvmw